In [ ]:
!pip install ultralytics

# Install Yolov8

In [ ]:
from ultralytics import YOLO
import os
from IPython.display import display,Image
from IPython import display
display.clear_output()
!yolo mode=checks

# Train the Yolov8 model

### Download the Data

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="put your API KEY")
project = rf.workspace("deep-learing-p9dxb").project("detection-drowsiness")
dataset = project.version(1).download("yolov8")


# EDA

In [ ]:
%%capture
# data-gradients (suitable for performing EDA on complex image dataset)
!pip install data-gradients

# for displaying pdfs as images
!pip install pdf2image
!apt-get install poppler-utils

# for pretty printing json
!pip install Pygments

In [ ]:
from PIL import Image
from pdf2image import convert_from_path
from IPython.display import display

def display_pdf_pages (pdf_path):
    """
    Display each page of a PDF file as images in separate output cells.
    Args:
    pdf_path (str): The path to the PDF file.
    Raises:
    FileNotFoundError: If the specified PDF file is not found.
    Returns:
    None
    """
    try:
        # Convert PDF to a list of PIL Images
        images = convert_from_path(pdf_path)

        # Display each image
        for i, image in enumerate(images):
          # Display the image
          display(image)

    except FileNotFoundError:
        raise FileNotFoundError("The specified PDF file was not found.")

The print_pretty_json function opens a JSON file, formats the data with an indent of 4 spaces using json.dumps, applies syntax highlighting with Pygments, and prints the pretty-printed JSON data in the cell below.

In [ ]:
import json
from pygments import highlight, lexers, formatters
from pathlib import Path

def print_pretty_json(file_path):
    """
    Function to pretty print a JSON file with colorization.
    Args:
    file_path (Union[str, Path]): The path of the JSON file to be pretty-printed.
    Raises:
    FileNotFoundError: If the file at 'file_path doesn't exist.
    json.JSONDecodeError: If the file at 'file_path is not valid JSON.
    """
    try:
      # Open the file
      with open(file_path, 'r') as f:
        # Load the JSON data from the file
        data= json.load(f)
    except FileNotFoundError:
      raise FileNotFoundError (f"File not found: {file_path}")
    except json.JSONDecodeError:
      raise json.JSONDecodeError("Invalid JSON file", "", 0)

    # Pretty print the JSON data with an indent of 4 spaces
    formatted_json = json.dumps (data, indent=4)

    # Colorize the pretty-printed JSON data
    colorful_json = highlight(formatted_json,
                              lexers.JsonLexer(),
                              formatters.TerminalFormatter()
                              )
    # Print the colorized, pretty-printed JSON data
    print(colorful_json)

In [ ]:
import yaml

# define path to YAML file
yaml_file_path = '/content/detection-drowsiness--1/data.yaml'

# open the YAML file and load it into a dictionary
with open(yaml_file_path, 'r') as f:
  data_yaml = yaml.safe_load(f)

In [ ]:
# Instantiating Dataloaders
dataset_params = {
    'data_dir':'/content/detection-drowsiness--1',
    'train_images_dir':'train/images',
    'train_labels_dir':'train/labels',
    'val_images_dir':'valid/images',
    'val_labels_dir':'valid/labels',
    'test_images_dir':'test/images',
    'test_labels_dir':'test/labels',
    'classes': ['awake', 'drowsy']
}

In [ ]:
from data_gradients.datasets.detection import YoloFormatDetectionDataset
train_set = YoloFormatDetectionDataset (root_dir="/content/detection-drowsiness--1",
                                        images_dir="train/images",
                                        labels_dir="train/labels")

val_set = YoloFormatDetectionDataset (root_dir="/content/detection-drowsiness--1",
                                      images_dir="valid/images",
                                      labels_dir="valid/labels")

In [ ]:
from data_gradients.managers.detection_manager import DetectionAnalysisManager
import matplotlib
matplotlib.use('Agg') # This line is only for Colab

analyzer = DetectionAnalysisManager(
    report_title="Testing Data-Gradients Object Detection",
    train_data=train_set,
    val_data=val_set,
    class_names=['awake', 'drowsy'],
)

analyzer.run()
# when running, input 0, then 0, then 1

In [ ]:
display_pdf_pages("/content/logs/Testing_Data-Gradients_Object_Detection/Report.pdf")

In [ ]:
# Acessing report in json format
# report_json = '/content/logs/Testing_Data-Gradients_Object_Detection/summary.json'
# print_pretty_json(report_json)

In [ ]:
def run_detection_analysis(report_title,
                           feature_extractors,
                           train_data,
                           val_data,
                           class_names):

    # create an instance of DetectionAnalysisManager
    analyzer = DetectionAnalysisManager(
      report_title=report_title,
      feature_extractors = feature_extractors,
      train_data=train_data,
      val_data=val_data,
      class_names = class_names
    )

    # Run the analysis
    analyzer.run()

In [ ]:
run_detection_analysis(report_title= 'DetectionBoundingBoxPerImageCount',
                        feature_extractors = 'DetectionBoundingBoxPerImageCount',
                        train_data=train_set,
                        val_data=val_set,
                        class_names=['awake', 'drowsy'])

In [ ]:
display_pdf_pages('/content/logs/DetectionBoundingBoxPerImageCount/Report.pdf')

In [ ]:
report_json = '/content/logs/DetectionBoundingBoxPerImageCount/summary.json'
print_pretty_json(report_json)

# The model

In [ ]:
!yolo task=detect mode=train model=yolov8s.pt data={dataset.location}/data.yaml epochs =15 imgsz = 640

In [ ]:
Image(filename =f'/content/runs/detect/train/confusion_matrix.png', width = 600)

In [ ]:
Image(filename= f'/content/runs/detect/train/results.png')

In [ ]:
!yolo task= detect mode=val model= /content/runs/detect/train/weights/best.pt data={dataset.location}/data.yaml

# The results

In [ ]:
!yolo task= detect mode=val model= /content/runs/detect/train/weights/best.pt data={dataset.location}/test.yaml

# show the test sample results

In [ ]:
!yolo task=detect mode= predict model= /content/runs/detect/train/weights/best.pt conf=0.5 source ={dataset.location}/test/images

In [ ]:
import glob
from IPython.display import Image, display
for image_path in glob.glob(f'/content/runs/detect/predict/*.jpg'):
  display(Image(filename= image_path ,height = 600))
  print("\n")


In [ ]:
import torch
import numpy as np
import cv2
from time import time
from ultralytics import YOLO
import matplotlib.pyplot as plt
import supervision as sv


class ObjectDetection:

    def __init__(self, capture_index):

        self.capture_index = capture_index
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        print("Using Device: ", self.device)
        self.model = self.load_model()
        self.CLASS_NAMES_DICT = self.model.model.names
        self.box_annotator = sv.BoxAnnotator(sv.ColorPalette.default(), thickness=3, text_thickness=3, text_scale=1.5)

    def load_model(self):

        model = YOLO("best.pt")  # load a pretrained YOLOv8n model
        model.fuse()

        return model

    def predict(self, frame):

        results = self.model(frame)

        return results

    def plot_bboxes(self, results, frame):

        xyxys = []
        confidences = []
        class_ids = []

        # Extract detections for person class
        for result in results:
            boxes = result.boxes.cpu().numpy()
            if len(boxes) > 0:
                class_id = boxes.cls[0]
                conf = boxes.conf[0]
                xyxy = boxes.xyxy[0]

                if class_id == 0.0:
                    xyxys.append(result.boxes.xyxy.cpu().numpy())
                    confidences.append(result.boxes.conf.cpu().numpy())
                    class_ids.append(result.boxes.cls.cpu().numpy().astype(int))

        # Setup detections for visualization
        detections = sv.Detections(
            xyxy=results[0].boxes.xyxy.cpu().numpy(),
            confidence=results[0].boxes.conf.cpu().numpy(),
            class_id=results[0].boxes.cls.cpu().numpy().astype(int),
        )

        # Assuming detections.xyxy, detections.confidence, detections.class_id, and detections.tracker_id are numpy arrays/lists or None
        xyxy = detections.xyxy if detections.xyxy is not None else []
        confidence = detections.confidence if detections.confidence is not None else []
        class_id = detections.class_id if detections.class_id is not None else []
        tracker_id = detections.tracker_id if detections.tracker_id is not None else []

        self.labels = [f"{self.CLASS_NAMES_DICT[class_id]} {conf.item():0.2f}"
                       for xy, conf, class_id, tracker_id in zip(xyxy, confidence, class_id, tracker_id)]

        # Annotate and display frame
        frame = self.box_annotator.annotate(scene=frame, detections=detections, labels=self.labels)

        return frame

    def __call__(self):
        cap = cv2.VideoCapture(self.capture_index)
        assert cap.isOpened()
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

        while True:
            start_time = time()

            ret, frame = cap.read()
            if not ret:  # Check if the frame was read successfully
                break  # Break the loop if there are no more frames

            results = self.predict(frame)
            frame = self.plot_bboxes(results, frame)

            end_time = time()
            fps = 1 / np.round(end_time - start_time, 2)

            plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            plt.title(f'YOLOv8 Detection - FPS: {int(fps)}')
            plt.axis('off')
            plt.pause(0.001)

        cap.release()
        plt.close()

# Create an instance of ObjectDetection
detector = ObjectDetection(capture_index=1)

# print(cv2.VideoCapture(0).isOpened())
# Run the object detection loop
detector()
